In [1]:
%%time
import os
import json
import warnings
from math import sqrt
import os
import ipfshttpclient
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from web3 import Web3
from keras import metrics
from keras.layers import GRU, Input, RepeatVector, concatenate, Activation, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from pandas import read_csv
from numpy import split, array
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error, mean_squared_log_error
from tensorflow.keras.models import Model, Sequential, model_from_json
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import RootMeanSquaredError, MeanSquaredError, MeanAbsoluteError
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_log_error
warnings.filterwarnings("ignore")

CPU times: total: 1.12 s
Wall time: 7.2 s


In [2]:
%%time
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8599"))
print(w3.is_connected())
w3.eth.default_account=w3.eth.accounts[0]

True
CPU times: total: 0 ns
Wall time: 18.5 ms


In [3]:
%%time
abi=[
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "accessListAddress",
				"type": "address"
			}
		],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "change_train_status",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "configsCount",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "deleteConfiguration",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getConfiguration",
		"outputs": [
			{
				"components": [
					{
						"internalType": "uint256",
						"name": "n_past",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "n_features",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "n_future",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "lstm_units_1",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "lstm_units_2",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "dense_units_1",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "dense_units_2",
						"type": "uint256"
					},
					{
						"internalType": "bool",
						"name": "train",
						"type": "bool"
					}
				],
				"internalType": "struct Config_Data.Configuration",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "getNodeAddress",
				"type": "address"
			}
		],
		"name": "getNodeInfo",
		"outputs": [
			{
				"internalType": "bool",
				"name": "",
				"type": "bool"
			},
			{
				"internalType": "uint8",
				"name": "",
				"type": "uint8"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "modelConfigs",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "n_past",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "n_features",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "n_future",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "lstm_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "lstm_units_2",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "dense_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "dense_units_2",
				"type": "uint256"
			},
			{
				"internalType": "bool",
				"name": "train",
				"type": "bool"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "_configsCount",
				"type": "uint256"
			},
			{
				"internalType": "bool",
				"name": "_train",
				"type": "bool"
			},
			{
				"internalType": "uint256",
				"name": "_n_past",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_n_features",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_n_future",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_lstm_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_lstm_units_2",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_dense_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_dense_units_2",
				"type": "uint256"
			}
		],
		"name": "setConfiguration",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			},
			{
				"internalType": "bool",
				"name": "_train",
				"type": "bool"
			},
			{
				"internalType": "uint256",
				"name": "_n_past",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_n_features",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_n_future",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_lstm_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_lstm_units_2",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_dense_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_dense_units_2",
				"type": "uint256"
			}
		],
		"name": "updateConfiguration",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	}
]


CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
%%time
address = "0x3F2948E1faCc1F5B394eeb0D61B83A5fF07aaEcc"
Conf_contract = w3.eth.contract(address=address, abi=abi)

CPU times: total: 0 ns
Wall time: 6.31 ms


In [5]:
%%time
abi1=[
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "modelCount",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_modelData1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_modelData2",
				"type": "string"
			}
		],
		"name": "setModelData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "dataCount",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_data1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_data2",
				"type": "string"
			}
		],
		"name": "setTrainingData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "accessListAddress",
				"type": "address"
			}
		],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "modelCount",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_modelData1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_modelData2",
				"type": "string"
			}
		],
		"name": "updateModelData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "dataCount",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_data1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_data2",
				"type": "string"
			}
		],
		"name": "updateTrainingData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "dataMap",
		"outputs": [
			{
				"internalType": "string",
				"name": "data1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "data2",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getModelData",
		"outputs": [
			{
				"components": [
					{
						"internalType": "string",
						"name": "modelData1",
						"type": "string"
					},
					{
						"internalType": "string",
						"name": "modelData2",
						"type": "string"
					}
				],
				"internalType": "struct Publish.ModelData",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "getNodeAddress",
				"type": "address"
			}
		],
		"name": "getNodeInfo",
		"outputs": [
			{
				"internalType": "bool",
				"name": "",
				"type": "bool"
			},
			{
				"internalType": "uint8",
				"name": "",
				"type": "uint8"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getTrainingData",
		"outputs": [
			{
				"components": [
					{
						"internalType": "string",
						"name": "data1",
						"type": "string"
					},
					{
						"internalType": "string",
						"name": "data2",
						"type": "string"
					}
				],
				"internalType": "struct Publish.TrainingData",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "modelMap",
		"outputs": [
			{
				"internalType": "string",
				"name": "modelData1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "modelData2",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]

CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
%%time
address1 = "0x6041cc1c53F338281FB623b409Ebd90e40557f71"
Pub_contract = w3.eth.contract(address=address1, abi=abi1)

CPU times: total: 15.6 ms
Wall time: 7.17 ms


In [7]:
%%time
# Connect to IPFS
client = ipfshttpclient.connect("/ip4/127.0.0.1/tcp/5001/http")

# Add model architecture and weights to IPFS
#res = client.add("model.json", "model.h5")

CPU times: total: 0 ns
Wall time: 26.6 ms


In [ ]:
#h=Conf_contract.functions.getConfiguration(1).call()

In [8]:
%%time
n_past=30
n_features=14
n_future=20

CPU times: total: 0 ns
Wall time: 0 ns


In [15]:
%%time
T_u=Pub_contract.functions.getTrainingData(3).call()

CPU times: total: 0 ns
Wall time: 9.34 ms


In [16]:
print(T_u)

('QmXe4J5uAzPNY3utHQv4xN7WExTimUcmZ39PEtoZocpFw5', 'QmXe4J5uAzPNY3utHQv4xN7WExTimUcmZ39PEtoZocpFw5')


In [17]:
%%time
# Retrieve file from IPFS
res1 = client.cat(T_u[1])

# Write the data to a file
with open('retrieved_data.json', 'wb') as f:
    f.write(res1)

CPU times: total: 3.31 s
Wall time: 14.8 s


In [18]:
%%time
# Read the JSON file
with open('retrieved_data.json', 'r') as f:
    data = json.load(f)

# Convert lists back into numpy arrays
X = np.array(data['x'])
y = np.array(data['y'])

CPU times: total: 19.8 s
Wall time: 24 s


In [19]:
H_u=Pub_contract.functions.getModelData(3).call()

In [20]:
np.shape(X),np.shape(y)

((99811, 30, 14), (99811, 20))

In [21]:
%%time
# Get hashes
model_json_hash_u = H_u[0]
model_h5_hash_u = H_u[1]


CPU times: total: 0 ns
Wall time: 0 ns


In [22]:
%%time

# Define the directory where you want to save the model and weights
save_dir = "C:\\Users\\montd\Desktop\\Project\\Deep Learning\\2\\Updatable3"
# Retrieve the files from IPFS
model_json_file_u = client.cat(model_json_hash_u)
model_h5_file_u = client.cat(model_h5_hash_u)
# Save the retrieved model architecture
with open(os.path.join(save_dir, 'retrieved_model_u.json'), 'w') as file:
    file.write(model_json_file_u.decode('utf-8'))

# Save the retrieved model weights
with open(os.path.join(save_dir, 'retrieved_model_u.h5'), 'wb') as file:
    file.write(model_h5_file_u)

# Load the model architecture
with open(os.path.join(save_dir, 'retrieved_model_u.json'), 'r') as file:
    loaded_model_json = file.read()

# Create a model from the loaded architecture
loaded_model_u = model_from_json(loaded_model_json)

# Load the weights into the model
loaded_model_u.load_weights(os.path.join(save_dir, 'retrieved_model_u.h5'))

CPU times: total: 312 ms
Wall time: 488 ms


In [23]:
loaded_model_u.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 128)              30720     
 l)                                                              
                                                                 
 repeat_vector (RepeatVector  (None, 20, 128)          0         
 )                                                               
                                                                 
 time_distributed (TimeDistr  (None, 20, 1)            129       
 ibuted)                                                         
                                                                 
Total params: 30,849
Trainable params: 30,849
Non-trainable params: 0
_________________________________________________________________


In [24]:
%%time
loaded_model_u.compile(optimizer='adam', 
                   loss='mean_squared_error', 
                   metrics=[RootMeanSquaredError(name='rmse'), 
                            MeanSquaredError(name='mse'), 
                            MeanAbsoluteError(name='mae')])

CPU times: total: 0 ns
Wall time: 34.1 ms


In [25]:

%%time
# Walk-forward validation parameters
n_splits = 5
n_train = X.shape[0] // n_splits
# Initialize performance metrics
mse_scores = []
mae_scores = []
r2_scores = []
nrmse_scores = []
rmse_scores = []
mase_scores = []
mape_scores = []
smape_scores = []

# Walk-forward validation loop
for i in range(n_splits):
    # Split data into train and test sets
    X_train = X[: n_train * (i + 1)]
    y_train = y[: n_train * (i + 1)]
    X_test = X[n_train * (i + 1): n_train * (i + 2)]
    y_test = y[n_train * (i + 1): n_train * (i + 2)]

    # Create input features and target values
   # X_train, y_train = create_dataset(X_train, y_train, n_past)
   # X_test, y_test = create_dataset(X_test, y_test, n_past)

    # Train the model
    loaded_model_u.fit(X_train, y_train, epochs=5, batch_size=64, verbose=0)

    # Generate predictions on test data
    predictions = loaded_model_u.predict(X_test)

    # Flatten the 3D arrays to 2D arrays
    y_test_flat = y_test.reshape(-1, 1)
    predictions_flat = predictions.reshape(-1, 1)

    # Calculate performance metrics
    mse = mean_squared_error(y_test_flat, predictions_flat)
    mae = mean_absolute_error(y_test_flat, predictions_flat)
    r2 = r2_score(y_test_flat, predictions_flat)
    rmse = np.sqrt(mse)
    nrmse = rmse / (y_test_flat.max() - y_test_flat.min())
    mean_absolute_error_naive = np.mean(np.abs(y_test_flat[1:] - y_test_flat[:-1]))
    mase = mae / mean_absolute_error_naive
    mape = mean_absolute_percentage_error(y_test_flat, predictions_flat)
    smape = mean_absolute_percentage_error(y_test_flat, predictions_flat, multioutput='uniform_average')

    # Append the scores to the corresponding lists
    mse_scores.append(mse)
    mae_scores.append(mae)
    r2_scores.append(r2)
    nrmse_scores.append(nrmse)
    rmse_scores.append(rmse)
    mase_scores.append(mase)
    mape_scores.append(mape)
    smape_scores.append(smape)

# Average performance metrics
avg_mse = np.mean(mse_scores)
avg_mae = np.mean(mae_scores)
avg_r2 = np.mean(r2_scores)
avg_nrmse = np.mean(nrmse_scores)
avg_rmse = np.mean(rmse_scores)
avg_mase = np.mean(mase_scores)
avg_mape = np.mean(mape_scores)
avg_smape = np.mean(smape_scores)

print("Average MSE:", avg_mse)
print("Average MAE:", avg_mae)
print("Average R-squared:", avg_r2)
print("Average NRMSE:", avg_nrmse)
print("Average RMSE:", avg_rmse)
print("Average MASE:", avg_mase)
print("Average MAPE:", avg_mape)
print("Average SMAPE:", avg_smape)


1/1 [==============================] - 0s 20ms/step
Average MSE: 2.115299339002641e-07
Average MAE: 0.00035700783644586483
Average R-squared: 0.7999517811126694
Average NRMSE: inf
Average RMSE: 0.00038270353731946493
Average MASE: inf
Average MAPE: 0.03465656764594745
Average SMAPE: 0.03465656764594745
CPU times: total: 5min 59s
Wall time: 4min 47s


In [26]:
import os

# Define the directory where you want to save the model and weights
save_dir = "C:\\Users\\montd\Desktop\\Project\\Deep Learning\\2\\Updatable"

In [27]:
%%time
# Save model architecture to JSON
model_json = loaded_model_u.to_json()
with open(os.path.join(save_dir, "model_u.json"), "w") as json_file:
    json_file.write(model_json)

# Save weights to HDF5
loaded_model_u.save_weights(os.path.join(save_dir, "model_u.h5"))

CPU times: total: 15.6 ms
Wall time: 21.3 ms


In [28]:
%%time
# Define the paths to the model architecture and weights files
file_names = ["model_u.json", "model_u.h5"]
file_paths = [os.path.join(save_dir, name) for name in file_names]

# Add each file to IPFS
res = []
for path in file_paths:
    res.append(client.add(path))

# Close the connection to the IPFS client
client.close()

CPU times: total: 15.6 ms
Wall time: 131 ms


In [29]:
%%time
print(res[0]['Hash'])
print(res[1]['Hash'])

QmNam2WWMSaKMYZg6FArv2gmTT7BZswcrXaoGpRZWH5QRL
QmXSqV3r4QFvYRBDnqjJiX8pyMHWxLfgPikeD12zWSHGTg
CPU times: total: 0 ns
Wall time: 982 µs


In [ ]:
#Pub_contract.functions.setModelData(4,res[0]['Hash'],res[1]['Hash']).transact()
#Pub_contract.functions.setModelData(3,res[0]['Hash'],res[1]['Hash']).transact()

In [30]:
%%time
Pub_contract.functions.updateModelData(3,res[0]['Hash'],res[1]['Hash']).transact()

CPU times: total: 0 ns
Wall time: 30.3 ms


HexBytes('0xac403f93fbf8322a0a7f52bc6d3097b0ed962271baf22b4d1b82880b77e6dfb6')

In [ ]:
#Pub_contract.functions.updateModelData(2,res[0]['Hash'],res[1]['Hash']).transact()